In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os

from mujoco_py import load_model_from_path, MjSim
from shared_utils.general import gen_models_folder_path
from utils import add_joint_pos, add_joint_vel, add_musc_force
from shared_utils.mujoco import n_step_forward

In [ ]:
PATH_TO_MODEL = os.path.join(
        gen_models_folder_path(), "muscles", "eval_elbow_flex_ext.xml")

model = load_model_from_path(PATH_TO_MODEL)
sim = MjSim(model)

orig_state = sim.get_state()

In [ ]:
elbow_joint_name = "el_x"
elbow_joint_index = model.joint_name2id(elbow_joint_name)

time_step = model.opt.timestep

In [ ]:
# experiment params
phases_nb_steps = [400, 400]
control_forces = [2.5, -1.5]
init_steps = 100

# initialize variables
elbow_pos = []
elbow_vel = []
elbow_musc_torque = []

sim.set_state(orig_state)
n_step_forward(init_steps,sim)

for control_force, phase_nb_steps in zip(control_forces, phases_nb_steps):
    sim.data.qfrc_applied[elbow_joint_index] = control_force

    for _ in range(phase_nb_steps):
        add_joint_pos(sim.data, elbow_pos, elbow_joint_name)
        add_joint_vel(sim.data, elbow_vel, elbow_joint_name)
        add_musc_force(sim.data, elbow_musc_torque, elbow_joint_index)

        sim.step()

time_array = time_step*np.arange(0, len(elbow_pos))
deg_elbow_pos = np.rad2deg(elbow_pos)

In [ ]:
fig, axes = plt.subplots(2, 2)
  
fig.set_size_inches(12, 10)
fig.suptitle('Elbow experiment')

# plot jangular displacement
axes[0,0].scatter(time_array[:phases_nb_steps[0]], deg_elbow_pos[:phases_nb_steps[0]], label="Flexion phase")
axes[0,0].scatter(time_array[phases_nb_steps[0]:], deg_elbow_pos[phases_nb_steps[0]:], label="Extension phase")

axes[0,0].set_xlabel("Time (s)")
axes[0,0].set_ylabel("Angular displacement (deg)")
axes[0,0].legend()
axes[0,0].set_title("Angular displacement")

# plot passive muscular torque
axes[0,1].scatter(deg_elbow_pos[:phases_nb_steps[0]],
                      elbow_musc_torque[:phases_nb_steps[0]], label="Flexion phase")
axes[0,1].scatter(deg_elbow_pos[phases_nb_steps[0]:],
                      elbow_musc_torque[phases_nb_steps[0]:], label="Extension phase")

axes[0,1].set_xlabel("Elbow flexion angle (deg)")
axes[0,1].set_ylabel("Passive muscular torque (N.m)")


axes[0,1].legend()
axes[0,1].set_title("Passive muscular torque evolution during experiment")

# flexion alone
axes[1,0].scatter(deg_elbow_pos[:phases_nb_steps[0]],
                      elbow_musc_torque[:phases_nb_steps[0]])

axes[1,0].vlines(0, np.min(elbow_musc_torque[:phases_nb_steps[0]]), np.max(elbow_musc_torque[:phases_nb_steps[0]]), colors='black', linestyles='dashed')
axes[1,0].vlines(120, np.min(elbow_musc_torque[:phases_nb_steps[0]]), np.max(elbow_musc_torque[:phases_nb_steps[0]]), colors='black', linestyles='dashed')

axes[1,0].set_xlabel("Elbow flexion angle (deg)")
axes[1,0].set_ylabel("Passive muscular torque (N.m)")

axes[1,0].set_title("Passive muscular torque evolution along flexion")

# extension alone
axes[1,1].scatter(deg_elbow_pos[phases_nb_steps[0]:],
                      elbow_musc_torque[phases_nb_steps[0]:], c=u'#ff7f0e')
axes[1,1].vlines(0, np.min(elbow_musc_torque[phases_nb_steps[0]:]), np.max(elbow_musc_torque[:phases_nb_steps[0]]), colors='black', linestyles='dashed')
axes[1,1].vlines(120, np.min(elbow_musc_torque[phases_nb_steps[0]:]), np.max(elbow_musc_torque[:phases_nb_steps[0]]), colors='black', linestyles='dashed')

axes[1,1].set_xlabel("Elbow flexion angle (deg)")
axes[1,1].set_ylabel("Passive muscular torque (N.m)")

axes[1,1].set_title("Passive torque evolution along extension")

plt.show()